<a href="https://colab.research.google.com/github/PilarAriasVazquez/Quantum-Machine-Learning/blob/main/VQC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VQC (Variatonal Quantum Circuits)

Main concepts:
-
- VQC: Quantum computing hybrid model that is design to work on actuals quantum computers (NISQ).
- Quantum Feature Map: Mathematical map that embeds classical data into a quantum state (qubits rotations)
- Ansatz: Basic Architecture of a circuit. Logic doors with  $\theta$ values to optimize
- Expectation Value: Measure with respect to the Z axis several times and we will return the value of the probability that it is 0 - the probability that it is 1 [P(0) - P(1)]. Equivalent to forward pass.
- Parameter Shift Rule: The quantums backpropagation.


Libraries:
- Pennylane: Open-source Python framework for quantum programming.

En esencia, un VQC es el equivalente a una capa de una Red Neuroanl, donde las neuronas son sustituidas por rotaciones de qubits. Los pesos y bias que se ajustan en Deep Learning clásico, es lo equivalente al ajuste de ángulos de las puertas lógicas cuánticas. Encontramos 3 bloques:
1. Encoding -> Se pasa de un vector de características, al espacio de Hilbert. Se aplican puertas de rotación donde el ángulo es igual al valor del dato. No tiene parámetros entrenables.
2. Ansatz -> Los parámetros que se entrenan $\theta$. Suelen estar entrelazadas mediante puertas CNOT. Así, gracias a este entrelazamiento, los qubits comparten información de una manera que las neuronas clásicas no pueden. Algunos de los puntos positivos de esta característica, se basan en encontrar correlaciones no locales, es decir, representar correlaciones extremadamente complejas. Así como los problemas con la dimensionalidad limitada, pues, graicas al entrelazamiento y superposición, mientras un sistema clásico representa uno de los 2^n estados posibles a la vez, en un sistema cuántico se puede llegar a explorar esos 2^n estados simultáneamente.
3. Medida -> Colapsa el estado cuántico en valores clásicos. El valor esperado (normalmente entre -1 y 1). Este es el valor que espera la función de pérdiuda estándar.

Es variacional pues gracias a un optimizador clásico, los parámetros $\theta$ iteran hasta minimizar la pérdida, no se resuelve una ecuación cuántica compleja del tirón. Es un bucle híbrido:
1. Hardware cuántico: Ejecuta el circuito con unos parámetros $\theta$ y mide el resultado.
2. Hardware clásico: Recibe el resultado, calcula el gradiente y actualiza $\theta$ para la siguiente iteración.


Es decir, los VQC son modelos de kernel implícitos, en los que al pasar los datos por el paso 1 (encoding), se proyectan a una dimensión infinita en el espacio de Hilbert, y el paso 2 busca el hiperplano óptimo en ese espacio masivo para clasificar los datos.

In [17]:
!pip install pennylane --upgrade -q

In [18]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch.autograd import Variable

In [19]:
# Dispositivo
dev = qml.device("default.qubit", wires = 2)
# Circuito cuántico
@qml.qnode(dev, interface = "torch")
def quantum_net(weights, data):
  # Encoding
  qml.RY(data[0], wires = 0)
  qml.RY(data[1], wires = 1)
  # Ansatz -> Capa entrenable
  qml.RX(weights[0], wires = 0)
  qml.RX(weights[1], wires = 1)
  # Entrelazamiento
  qml.CNOT(wires = [0,1])
  # Medida
  return qml.expval(qml.PauliZ(0))

In [22]:
# Integración en el modelo Pytorch
class HybridModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    # inicializamos 2 pesos aleatorios
    self.weights = torch.nn.Parameter(torch.rand(2))
  def forward(self, x):
    return quantum_net(self.weights, x)

model = HybridModel()
dummy_data = torch.tensor([0.5, -1.2])
prediction = model(dummy_data)
print(f"Predicción cuántica inicial: {prediction.item():.4f}")



Predicción cuántica inicial: 0.8513
